In [ ]:
import time
import math
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys

# 크롬드라이버 열기
def chromeWebdriver():
    chrome_service = Service(executable_path=ChromeDriverManager().install())
    options =Options()
    options.add_experimental_option('detach',True)
    options.add_experimental_option('excludeSwitches', ['enable-logging'])

    driver = webdriver.Chrome(service=chrome_service,options=options)
    return driver

# 문자열 정리 함수
def cleaner(string):
    temp = string.replace('\n', '')
    temp = temp.replace('\r', '')
    temp = temp.replace('\t', '')
    return temp

JUMP = 2                                                                                            # sleep 시간 

In [ ]:
# 키워드 입력받기
#keyword = input('키워드를 입력하세요:')
keyword = '뭉티기'

# 수집 개수 입력받기
goal = 300
# 웹사이트 열기
browser = chromeWebdriver()
browser.get('https://shopping.naver.com/home/p/index.naver')                                            # 네이버 쇼핑 페이지
browser.implicitly_wait(5)                                                                              # 로딩이 끝날 때까지 10초까지는 기다려줌

#검색창 클릭
search = browser.find_element(By.CSS_SELECTOR,'input._searchInput_search_input_QXUFf')
search.click()

#검색어 입력
search.send_keys(keyword)
search.send_keys(Keys.ENTER)

In [ ]:
eighty_btn = browser.find_elements(By.CSS_SELECTOR, 'a.subFilter_btn_select__K6F79')[1]
browser.execute_script("arguments[0].click();",eighty_btn)

eighty_btn2 = browser.find_element(By.CSS_SELECTOR, '#__next > div > div.style_container__1YjHN > div.style_inner__18zZX > div.style_content_wrap__1PzEo > div.style_content__2T20F > div.seller_filter_area > div > div.subFilter_sort_choice__1SFXG > div:nth-child(3) > ul > li:nth-child(4) > a')
browser.execute_script("arguments[0].click();",eighty_btn2)

In [ ]:
total_num = int(browser.find_element(By.CSS_SELECTOR, 'span.subFilter_num__2x0jq').text)
total_page = math.ceil(total_num /80 *10)

In [ ]:
dic_list =[]
rank = 1 

In [ ]:
def scroll():
    before_h = browser.execute_script("return window.scrollY")                  # 0으로 초기화하기

    #무한 스크롤
    while True:
        # 맨 아래로 스크롤을 내린다
        browser.find_element(By.CSS_SELECTOR, 'body').send_keys(Keys.END)

        #스크롤 사이 페이지 로딩 시간
        time.sleep(1)

        #스크롤 후 높이
        after_h = browser.execute_script('return window.scrollY')

        if after_h == before_h:
            break
        before_h = after_h

In [ ]:
boxes = browser.find_elements(By.CSS_SELECTOR,'li.basicList_item__2XT81')
box = boxes[0]
bbox = box.find_elements(By.CSS_SELECTOR,'em.basicList_option__3eF2s')[0].text
print(bbox.split()[1])


In [ ]:
type(box.get_attribute('class'))
print(int(box.find_element(By.CSS_SELECTOR,'span.price_num__2WUXn').text[:-1].replace(',','')))

In [ ]:
# for pg in range(total_page):
#     try:
#         for i in range(2,11):
#             scroll()
#             boxes = browser.find_elements(By.CSS_SELECTOR,'li.basicList_item__2XT81')
#             browser.implicitly_wait(JUMP)     
#             for box in boxes:
#                 if 'ad' not in box.get_attribute('class'):                                                      # 광고 상품 패스하기
#                     data_dic[rank]={}
#                     # 제목, 가격 추출
#                     data_dic[rank]['상품명'] = box.find_element(By.CSS_SELECTOR, 'a.basicList_link__1MaTN').text
#                     data_dic[rank]['링크'] = box.find_element(By.CSS_SELECTOR, 'a.basicList_link__1MaTN').get_attribute('href')
#                     data_dic[rank]['가격'] = int(box.find_element(By.CSS_SELECTOR,'span.price_num__2WUXn').text[:-1].replace(',',''))     
#                     browser.implicitly_wait(JUMP)     
#                     # 리뷰, 구매건수, 찜하기 추출
#                     r_boxes = box.find_elements(By.CSS_SELECTOR,'a.basicList_etc__2uAYO')                      
#                     browser.implicitly_wait(JUMP)     
#                     r_boxes.append(box.find_element(By.CSS_SELECTOR,'button.basicList_btn_zzim__2MGkM'))
#                     browser.implicitly_wait(JUMP)     
#                     for item in r_boxes:
#                         num = item.find_element(By.TAG_NAME, 'em').text
#                         browser.implicitly_wait(JUMP)     
#                         temp = item.text
#                         temp = temp[:len(temp)-len(num)]
#                         data_dic[rank][temp] = int(num.replace(',',''))

#                     # 쇼핑몰 추출
#                     sboxes = box.find_elements(By.CSS_SELECTOR,'a.basicList_mall__sbVax')
#                     for sbox in sboxes:
#                         if sbox.text == '':
#                             shop = sbox.find_element(By.TAG_NAME,'img').get_attribute('alt')
#                             data_dic[rank]['스마트스토어'] = 0
#                             browser.implicitly_wait(JUMP)     
#                         else :
#                             shop = sbox.text
#                             data_dic[rank]['스마트스토어'] = 1

#                         data_dic[rank]['쇼핑몰'] = shop
                    
#                     # 배송비 추출
#                     bbox = box.find_elements(By.CSS_SELECTOR,'em.basicList_option__3eF2s')[0].text
#                     bbox = bbox.split()[1]
#                     if bbox == '무료':
#                         data_dic[rank]['배송비'] = 0
#                     else:
#                         bbox = int(bbox[:-1].replace(',',''))
#                         data_dic[rank]['배송비'] = bbox

#                     # 상세정보 초기화                    
#                     data_dic[rank]['용도'] = ''
#                     data_dic[rank]['부위'] = ''
#                     data_dic[rank]['중량'] = ''
#                     data_dic[rank]['등급'] = ''
                    
#                     # 상품 상세페이지 새창에 열었다 닫기
#                     if data_dic[rank]['스마트스토어'] == 1:
#                         browser.execute_script('window.open(\''+data_dic[rank]['링크']+'\');')
#                         time.sleep(1)
#                         browser.switch_to.window(window_name= browser.window_handles[-1])
#                         time.sleep(1)
#                         try:
#                             product_box = browser.find_elements(By.CSS_SELECTOR,'table._1_UiXWHt__')[1]
#                             product_info = product_box.text
#                             if '용도' in product_info:
#                                 tag_list = list(map(lambda x: x.text ,product_box.find_elements(By.CLASS_NAME,'_1iuv6pLHMD')))
#                                 info_list = list(map(lambda x: x.text ,product_box.find_elements(By.CLASS_NAME,'ABROiEshTD')))
#                                 print(tag_list,info_list)
#                                 for i in range(len(tag_list)):
#                                     data_dic[rank][tag_list[i]] = info_list[i] 
#                         except :
#                             data_dic[rank]['스마트스토어'] = 0
                        
#                         browser.close()
#                         time.sleep(1)
#                         browser.switch_to.window(window_name= browser.window_handles[0])

#                     print(data_dic[rank])
#                     print ()
#                     if rank == goal:
#                         break
#                     else: 
#                         rank +=1
#             try: 
#                 if rank == goal:
#                     break
#                 browser.find_element(By.CSS_SELECTOR,f'#__next > div > div.style_container__1YjHN > div.style_inner__18zZX > div.style_content_wrap__1PzEo > div.style_content__2T20F > div.pagination_pagination__6AcG4 > div > a:nth-child({i})').click()
#                 browser.implicitly_wait(JUMP)        
#             except Exception as e:
#                 pass
#         if rank == goal:
#             break
#     except:
#         pass


In [ ]:
for pg in range(total_page):
    try:
        for i in range(2,11):
            scroll()
            boxes = browser.find_elements(By.CSS_SELECTOR,'li.basicList_item__2XT81')
            browser.implicitly_wait(JUMP)     
            for box in boxes:
                if 'ad' not in box.get_attribute('class'):                                                      # 광고 상품 패스하기
                    # 제목, 가격 추출
                    data_dic = {}
                    data_dic['상품명'] = box.find_element(By.CSS_SELECTOR, 'a.basicList_link__1MaTN').text
                    data_dic['링크'] = box.find_element(By.CSS_SELECTOR, 'a.basicList_link__1MaTN').get_attribute('href')
                    data_dic['가격'] = int(box.find_element(By.CSS_SELECTOR,'span.price_num__2WUXn').text[:-1].replace(',',''))     
                    browser.implicitly_wait(JUMP)     
                    # 리뷰, 구매건수, 찜하기 추출
                    r_boxes = box.find_elements(By.CSS_SELECTOR,'a.basicList_etc__2uAYO')                      
                    browser.implicitly_wait(JUMP)     
                    r_boxes.append(box.find_element(By.CSS_SELECTOR,'button.basicList_btn_zzim__2MGkM'))
                    browser.implicitly_wait(JUMP)     
                    for item in r_boxes:
                        num = item.find_element(By.TAG_NAME, 'em').text
                        browser.implicitly_wait(JUMP)     
                        temp = item.text
                        temp = temp[:len(temp)-len(num)]
                        data_dic[temp] = int(num.replace(',',''))

                    # 쇼핑몰 추출
                    sboxes = box.find_elements(By.CSS_SELECTOR,'a.basicList_mall__sbVax')
                    for sbox in sboxes:
                        if sbox.text == '':
                            shop = sbox.find_element(By.TAG_NAME,'img').get_attribute('alt')
                            data_dic['스마트스토어'] = 0
                            browser.implicitly_wait(JUMP)     
                        else :
                            shop = sbox.text
                            data_dic['스마트스토어'] = 1

                        data_dic['쇼핑몰'] = shop
                    
                    # 배송비 추출
                    bbox = box.find_elements(By.CSS_SELECTOR,'em.basicList_option__3eF2s')[0].text
                    bbox = bbox.split()[1]
                    if bbox == '무료':
                        data_dic['배송비'] = 0
                    else:
                        bbox = int(bbox[:-1].replace(',',''))
                        data_dic['배송비'] = bbox

                    # 상세정보 초기화                    
                    data_dic['용도'] = ''
                    data_dic['부위'] = ''
                    data_dic['중량'] = ''
                    data_dic['등급'] = ''
                    
                    # 상품 상세페이지 새창에 열었다 닫기
                    if data_dic['스마트스토어'] == 1:
                        browser.execute_script('window.open(\''+data_dic['링크']+'\');')
                        time.sleep(1)
                        browser.switch_to.window(window_name= browser.window_handles[-1])
                        time.sleep(1)
                        try:
                            product_box = browser.find_elements(By.CSS_SELECTOR,'table._1_UiXWHt__')[1]
                            product_info = product_box.text
                            if '용도' in product_info:
                                tag_list = list(map(lambda x: x.text ,product_box.find_elements(By.CLASS_NAME,'_1iuv6pLHMD')))
                                info_list = list(map(lambda x: x.text ,product_box.find_elements(By.CLASS_NAME,'ABROiEshTD')))
                                print(tag_list,info_list)
                                for i in range(len(tag_list)):
                                    data_dic[tag_list[i]] = info_list[i] 
                        except :
                            try:
                                product_box = browser.find_elements(By.CSS_SELECTOR,'table.TH_yvPweZa')[1]
                                product_info = product_box.text
                                if '용도' in product_info:
                                    tag_list = list(map(lambda x: x.text ,product_box.find_elements(By.CLASS_NAME,'_15qeGNn6Dt')))
                                    info_list = list(map(lambda x: x.text ,product_box.find_elements(By.CLASS_NAME,'jvlKiI0U_y')))
                                    print(tag_list,info_list)
                                    for i in range(len(tag_list)):
                                        data_dic[tag_list[i]] = info_list[i] 
                            except:
                                data_dic['스마트스토어'] = 0
                        
                        browser.close()
                        time.sleep(1)
                        browser.switch_to.window(window_name= browser.window_handles[0])
                    print(data_dic)
                    dic_list.append(data_dic)
                    print ()
                    if rank == goal:
                        break
                    else: 
                        rank +=1
            try: 
                if rank == goal:
                    break
                browser.implicitly_wait(JUMP)   
                browser.find_element(By.CSS_SELECTOR,'a.pagination_next__1ITTf').click()
                browser.implicitly_wait(JUMP)        
            except Exception as e:
                pass
        if rank == goal:
            break
    except:
        pass


In [ ]:
df = pd.DataFrame(dic_list,columns=['상품명','가격','리뷰','구매건수','찜하기','스마트스토어','쇼핑몰','배송비','용도','부위','중량','등급','링크'])
df.index +=1
df.drop_duplicates()
df.to_csv('./뭉티기test2.csv',encoding='utf-8-sig')      

In [ ]:
print(df)